<a href="https://colab.research.google.com/github/zubair9703/Google-Earth-Engine-Random-Forest/blob/main/RF_on_RE3_SWIR_ipynb_SWIR2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 30.9 MB/s 
     |████████████████████████████████| 1.3 MB 47.1 MB/s 
     |████████████████████████████████| 3.0 MB 39.4 MB/s 
     |████████████████████████████████| 99 kB 10.9 MB/s 
     |████████████████████████████████| 1.2 MB 39.8 MB/s 
     |████████████████████████████████| 130 kB 62.9 MB/s 
     |████████████████████████████████| 98 kB 9.2 MB/s 
     |████████████████████████████████| 187 kB 60.1 MB/s 
     |████████████████████████████████| 3.3 MB 31.2 MB/s 
     |████████████████████████████████| 95 kB 3.6 MB/s 
     |████████████████████████████████| 46 kB 3.7 MB/s 
     |████████████████████████████████| 8.8 MB 63.2 MB/s 
     |████████████████████████████████| 128 kB 57.6 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 9.7 MB 40.7 MB/s 
     |████████████████████████████████| 

In [1]:
import ee
import geemap

In [2]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
s2=ee.ImageCollection("COPERNICUS/S2_SR")
roi=ee.Geometry.Polygon([[[77.45036134046319, 13.177235646600929],[77.45036134046319, 13.081444893977725],[77.5583363465667, 13.081444893977725],[77.5583363465667, 13.177235646600929]]])
filtered= s2.filter(ee.Filter.date('2022-01-01','2022-04-25')).filter(ee.Filter.bounds(roi)).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', .5)).first().clip(roi)

In [4]:
Map.setCenter(77.49525079053643,13.12339059592214,12)
Map.addLayer(filtered,{"bands":['B7','B11','B12'],"min":51,"max":4626},'Fcc')
Map

Map(center=[13.12339059592214, 77.49525079053643], controls=(WidgetControl(options=['position', 'transparent_b…

In [5]:
gcp=ee.FeatureCollection("projects/ee-zubahme97/assets/finaltraning_set")
bands= ['B7','B11','B12']
sample= filtered.select(bands).sampleRegions(**{"collection":gcp,"properties":['landcover'],"scale":10})
sample = sample.randomColumn()
trainingGcp = sample.filter(ee.Filter.lt('random', 0.7));
validationGcp = sample.filter(ee.Filter.gte('random', 0.7));


In [6]:
classifier = ee.Classifier.smileRandomForest(100).train(**{"features": trainingGcp,"classProperty": 'landcover',"inputProperties": bands})

In [7]:
classified= filtered.classify(classifier)

In [8]:
Map.addLayer(classified, {"min": 0, "max": 10, "palette": ['c81f04',  '100eff', '179750','ad7b0d','004902','ebff3f','ee6aff',"aaff5a","a6adba",'ba81a6','61ff4e']}, '2022')
legend_dict = {
    '0)Built-up': 'c81f04',
    '1)water': '100eff',
    '2)otherplantations': '179750',
    '3)coconut': 'ad7b0d',
    '4)mango': '004902',
    '5)openland': 'ebff3f',
    '6)fallowlands': 'ee6aff',
    '7)cropland': 'aaff5a',
    '8)scrublands': 'a6adba',
    '9)Eucalyptus': 'ba81a6',
    '10)Forest':'61ff4e'
}
# add_north_arrow(ax, text='N', xy=(0.1, 0.1), arrow_length=0.1, text_color='black', arrow_color='black', fontsize=20, width=5, headwidth=15, ha='center', va='center')
Map.add_legend(title="Classification of RE3,SWIR1 and SWIR2", legend_dict=legend_dict,position="bottomright")
Map

Map(center=[13.12339059592214, 77.49525079053643], controls=(WidgetControl(options=['position', 'transparent_b…

In [9]:
train_accuracy = classifier.confusionMatrix()
train_accuracy.getInfo()
train_accuracy.accuracy().getInfo()
train_accuracy.kappa().getInfo()
train_accuracy.producersAccuracy().getInfo()
train_accuracy.consumersAccuracy().getInfo()

[[1,
  1,
  1,
  0.9961089494163424,
  1,
  1,
  1,
  1,
  0.9955357142857143,
  0.9893617021276596,
  1]]

In [10]:
validated = validationGcp.classify(classifier)

In [11]:
testConfusionMatrix = validated.errorMatrix('landcover', 'classification')
print('Confusion Matrix')
testConfusionMatrix.getInfo()

Confusion Matrix


[[90, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 306, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 122, 1, 2, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 105, 0, 0, 0, 0, 0, 2, 0],
 [0, 0, 0, 0, 167, 0, 0, 2, 0, 5, 0],
 [0, 0, 0, 0, 0, 21, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 55, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 59, 0, 0, 0],
 [0, 0, 0, 0, 4, 0, 0, 0, 97, 2, 0],
 [0, 0, 0, 3, 2, 0, 0, 0, 1, 191, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 48]]

In [12]:
print('Test Accuracy')
testConfusionMatrix.accuracy().getInfo()

Test Accuracy


0.9760061919504643

In [13]:
print('kappa')
testConfusionMatrix.kappa().getInfo()

kappa


0.9723732821247161

In [14]:
print('ProdcuerAccuracy')
testConfusionMatrix.producersAccuracy().getInfo()

ProdcuerAccuracy


[[1],
 [1],
 [0.976],
 [0.9813084112149533],
 [0.9597701149425287],
 [0.9545454545454546],
 [1],
 [1],
 [0.941747572815534],
 [0.9695431472081218],
 [0.8888888888888888]]

In [15]:
print('UserAccuracy')
testConfusionMatrix.consumersAccuracy().getInfo()

UserAccuracy


[[1,
  1,
  1,
  0.963302752293578,
  0.9542857142857143,
  1,
  0.9821428571428571,
  0.9672131147540983,
  0.9897959183673469,
  0.9271844660194175,
  1]]

In [16]:
import os
import csv

In [17]:
os.chdir("/content/drive/MyDrive/ee")
home="/content/drive/MyDrive/ee"

In [18]:

out_dir = os.path.join(home, 'Downloads')
training_csv = os.path.join(out_dir, 'train_accuracy.csv')
testing_csv = os.path.join(out_dir, 'finalfcc2.csv')

# with open(training_csv, "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerows(train_accuracy.getInfo())

with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(testConfusionMatrix.getInfo())

In [ ]:
# out_dir = os.path.join(home, 'Downloads')
# training_csv = os.path.join(out_dir, 'pa.csv')
# testing_csv = os.path.join(out_dir, 'ua.csv')

# with open(training_csv, "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerows(testConfusionMatrix.producersAccuracy().getInfo())

# with open(testing_csv, "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerows(testConfusionMatrix.consumersAccuracy().getInfo())